# Climate CO₂ Forecast

This notebook demonstrates a machine learning approach to forecast CO₂ emissions in support of **SDG 13: Climate Action**.

## IMPORT THE RELEVANT LIBRARIES

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
